# Кластеризация ирисов Фишера

In [56]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import optimize
import random as rd
import requests
import io

In [57]:
data_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
raw_dataset = requests.get(data_url)
assert raw_dataset.status_code == 200

In [58]:
frame = pd.read_csv(io.StringIO(raw_dataset.text),header = None, quotechar=',', skipinitialspace=True)

In [97]:
dat_mat = np.zeros((150,4))
for i in range(4):
    dat_mat[:,i] = frame[i] 
dat_mat = dat_mat.T

In [98]:
dat_mat_train = np.zeros((4,75))
dat_mat_test = np.zeros((4,75))
for i in range(75):
    dat_mat_train[:,i] = dat_mat[:,i * 2]
    dat_mat_test[:,i] = dat_mat[:,1 + i * 2]


In [99]:
solution = np.zeros((3, 75))
solution[0,:25] = 1
solution[1,25:50] = 1
solution[2,50:75] = 1
#TODO: Сделать чуть более общим

In [100]:
NUM_INPUT = 4
NUM_HIDDEN = 4
NUM_OUTPUT = 3
#количество нейронов скрытого слоя

In [101]:
def logreg(x):
    return 1./(1.+np.exp(-x))

In [102]:
def train_network(wghs):
    wgs_mat_size_1 = NUM_INPUT * NUM_HIDDEN
    wgs_mat_size_2 = NUM_HIDDEN * NUM_OUTPUT
    wgs_1 = wghs[:wgs_mat_size_1].reshape((NUM_HIDDEN_NEURONS, 4))
    wgs_2 = wghs[wgs_mat_size_1:wgs_mat_size_1 + wgs_mat_size_2].reshape((3, NUM_HIDDEN))
    z = logreg(np.dot(wgs_1, dat_mat_train))
    z = logreg(np.dot(wgs_2, z))
    return z

In [103]:
def test_network(wghs, dat):
    dat = dat.reshape((4, 75))
    wgs_mat_size_1 = NUM_INPUT * NUM_HIDDEN
    wgs_mat_size_2 = NUM_HIDDEN * NUM_OUTPUT
    wgs_1 = wghs[:wgs_mat_size_1].reshape((NUM_HIDDEN_NEURONS, 4))
    wgs_2 = wghs[wgs_mat_size_1:wgs_mat_size_1 + wgs_mat_size_2].reshape((3, NUM_HIDDEN))
    z = logreg(np.dot(wgs_1, dat))
    z = logreg(np.dot(wgs_2, z))
    return z

In [104]:
def loss(wgs, data):
    z = test_network(wgs, data)
    _sum = 0
    for i in range(75):
        sq = np.linalg.norm(z[:,i] - solution[:,i])
        _sum += sq ** 2
    return _sum*(1./150.)    

In [113]:
wgs_size = 7 * NUM_HIDDEN_NEURONS
wgs_0 = np.random.sample(wgs_size)
while True:
    if optimize.minimize(loss, wgs_0, dat_mat_train).fun < 0.3:
        wgs_1 = optimize.minimize(loss, wgs_0, dat_mat_train).x
        break

In [120]:
a = test_network(wgs_1, dat_mat_test)
a = a.T
for i in range(75):
    for j in range(3):
        a[i,j] = (a[i,j]/np.sum(a[i,:])) 
for i in range(75):
    b = a[i,:]
    # print(np.argmax(b) + 1) 

##### Todo: Добавить графиков и по возможности сравнить с "KNN"